In [1]:
import numpy as np

with open('../../postprocessing/representation.npy', 'rb') as f:
    rep = np.load(f)

In [3]:
from sklearn.cluster import mean_shift

cluster_centers, labels = mean_shift(rep, cluster_all=False)

KeyboardInterrupt: 